In [1]:
# Install packages
install.packages("caret")

# Load required library
library(caret)

NameError: name 'install' is not defined

In [ ]:
# 2) DATA COLLECTION

# Read the Airbnb dataset for Seattle location
listings <- read.csv("listings.csv", stringsAsFactors = FALSE)

# View basic structure and first few rows
str(listings)
head(listings)


'data.frame':	6862 obs. of  79 variables:
 $ id                                          : num  6606 9419 9596 11012 25002 ...
 $ listing_url                                 : chr  "https://www.airbnb.com/rooms/6606" "https://www.airbnb.com/rooms/9419" "https://www.airbnb.com/rooms/9596" "https://www.airbnb.com/rooms/11012" ...
 $ scrape_id                                   : num  2.03e+13 2.03e+13 2.03e+13 2.03e+13 2.03e+13 ...
 $ last_scraped                                : chr  "6/22/2025" "6/22/2025" "6/22/2025" "6/22/2025" ...
 $ source                                      : chr  "city scrape" "city scrape" "previous scrape" "city scrape" ...
 $ name                                        : chr  "Fab, private seattle urban cottage!" "Glorious sun room w/ memory foambed" "the down home , spacious, central and fab!" "the orange house, quiet 'n central" ...
 $ description                                 : chr  "This tiny cottage is only 15x10, but it has everything you need..includi

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,⋯,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>
1,6606,https://www.airbnb.com/rooms/6606,2.02506e+13,6/22/2025,city scrape,"Fab, private seattle urban cottage!","This tiny cottage is only 15x10, but it has everything you need..including a garden just outside! Newer queen bed, down pillows/ duvet, tv, closet w iron and mini board, large dresser, fold down table and chair. The kitchenette has micro, toaster oven, mini fridge, coffee, tea n snacks! ( hot plate too under sink) The combo shower, toilet,European style is across from kitchen separated by a TALL shower curtain. <br />Our walking score is 95, with 2 neighborhoods w/in a mile, buses on corner.","A peaceful yet highly accessible neighborhood, gardens galore. The university district, Fremont, and wallingford are all walkable.( 5 Blocks from wallingford's business district.)",https://a0.muscache.com/pictures/45742/21116d7d_original.jpg,14942,⋯,4.77,4.88,4.57,str-opli-19-002622,f,3,3,0,0,0.83
2,9419,https://www.airbnb.com/rooms/9419,2.02506e+13,6/22/2025,city scrape,Glorious sun room w/ memory foambed,"This beautiful double room features sun filled oasis (room darkening shades provided) with a sleeping room (Full size bed) wardrobe and a second room that features a day bed, work desk and your own refrigerator. On a major bus line (124) . Room is located above a pool hall and two bars / restaurants (good food) We are in a creative arts district just minutes away from downtown Seattle. Great place for a couple, parents (day bed is big enough for a full grown person too. We have A/C and heat","Lots of restaurants (see our guide book) bars,coffee shops and antique /eclectic stores including the Georgetown Trailer Park mall filled with curiosities (open on weekends)We also have a huge parking lot and busses right across the street. This neighborhood is still a tiny bit gritty but also very safe and honestly more sparkly than a few years ago. Come see it for yourself<br />Industrial Arts district meets thriving commercial meets victorian era esthetics<br /><br />A little grit,a lot of love and a burgeoning arts community. Easy 15 minute bus ride to Pike Place Market,the Big Wheel and Seattle Center and transportation to points beyond like Ballard locks,university district and Bellevue<br />A little grit, and allot of love have resurrected this neighborhood and arts community. Easy 15 minute bus ride to Pike Place Market, the Big Wheel, the waterfront parks and the Seattle Center and transportation to points beyond like Ballard locks, University district and Bellevue<br />#Cli",https://a0.muscache.com/pictures/56645186/e5fb34c9_original.jpg,30559,⋯,4.89,4.68,4.67,Exempt,f,10,0,10,0,1.17
3,9596,https://www.airbnb.com/rooms/9596,2.02506e+13,6/22/2025,previous scrape,"the down home , spacious, central and fab!","We are in a great neighborhood, quiet, full of gardens, and walkability score ,94! Walk to downtown wallingford w/ its 20+ restaurants, movies, shops just 5 blocks away. we are 6 blk.s n. of gasworks park and the burke gilman trail,...Or you can walk .8 of a mile to Fremont,2 miles to downtown, 2 miles to the university district, ethnic restaurants, book stores and thrifting galore... like I said, central!","if you arrive early for check in at 3, I recommend Archie Mcfee's for joke gifts for all occasions! ... and Tutti Bella is fab pizza both blocks away 44 n stone, 45th and stone. There is always Gasworks park,( great views) and the essential bakery for lunch ,34th and woodlawn...a picnic?!",https://a0.muscache.com/picture

In [ ]:
# 3) SELECT RELEVANT COLUMNS

# Keep only useful features for price prediction
listings <- listings[, c("id", "price", "neighbourhood", "property_type",
                         "room_type", "bedrooms", "bathrooms", "amenities",
                         "host_since", "host_is_superhost", "number_of_reviews",
                         "review_scores_rating", "availability_30")]

In [ ]:
# 4) DATA CLEANING

# 4.1 Remove duplicate listings
listings <- listings[!duplicated(listings$id), ]

In [ ]:
# 4.2 Remove rows with missing price values
listings <- listings[!is.na(listings$price), ]

In [ ]:
# 4.3 Convert 'price' column from string to numeric
listings$price <- as.numeric(gsub("[\\$,]", "", listings$price))

In [ ]:
# 4.4 Remove unrealistic outliers (keep prices between $10 and $1000 per night)
listings <- listings[listings$price > 10 & listings$price < 1000, ]

In [ ]:
# 4.5 Handle missing numeric values by replacing with median
listings$number_of_reviews[is.na(listings$number_of_reviews)] <-
    median(listings$number_of_reviews, na.rm = TRUE)

listings$review_scores_rating[is.na(listings$review_scores_rating)] <-
    median(listings$review_scores_rating, na.rm = TRUE)

In [ ]:
# 4.6 Handle missing categorical values by replacing with 'Unknown'
listings$host_is_superhost[is.na(listings$host_is_superhost)] <- "Unknown"

In [ ]:
# 5) FEATURE ENGINEERING

# 5.1 Total number of amenities
listings$amenities_count <- sapply(strsplit(listings$amenities, ","), length)


In [ ]:
# 5.2 Host experience in years
# Assuming the date format is month/day/year, e.g., "4/26/2009"
listings$host_since <- as.Date(listings$host_since, format = "%m/%d/%Y")
listings$host_experience_years <- as.numeric(difftime(Sys.Date(),
                                                     listings$host_since,
                                                     units = "days")) / 365
# Replace NA values with 0
listings$host_experience_years[is.na(listings$host_experience_years)] <- 0

In [ ]:
# 5.3 Price per bedroom (avoid division by zero)
listings$price_per_bedroom <- listings$price / pmax(listings$bedrooms, 1)

In [ ]:
# 6) ENCODING CATEGORICAL VARIABLES

## Convert to factors
listings$property_type <- as.factor(listings$property_type)
listings$neighbourhood <- as.factor(listings$neighbourhood)
listings$room_type <- as.factor(listings$room_type)
listings$host_is_superhost <- as.factor(listings$host_is_superhost)

## One-hot encoding (dummy variables)
dummy_vars <- dummyVars(" ~ .", data = listings, fullRank = TRUE)
listings_encoded <- data.frame(predict(dummy_vars, newdata = listings))


In [ ]:
# 7) NORMALIZATION / SCALING

# Scale numeric features (optional - useful for some models like linear regression)
num_cols <- sapply(listings_encoded, is.numeric)
listings_encoded[num_cols] <- scale(listings_encoded[num_cols])

In [ ]:
# 8) TRAIN-TEST SPLIT

set.seed(123) # for reproducibility
train_index <- createDataPartition(listings_encoded$price, p = 0.8, list = FALSE)
train_data <- listings_encoded[train_index, ]
test_data <- listings_encoded[-train_index, ]

# Final check
dim(train_data)
dim(test_data)

[1] 4707 5696

[1] 1173 5696

In [ ]:
  # -------------------------------
  # 2) EXPLORATORY DATA ANALYSIS (EDA) & FEATURE SELECTION
  # -------------------------------

  # 2.1 Price distribution
  png("price_distribution.png", width = 800, height = 600)
  ggplot(listings, aes(x = price)) +
    geom_histogram(bins = 50, fill = "skyblue", color = "black") +
    theme_minimal() +
    ggtitle("Price Distribution") +
    xlab("Price ($)") + ylab("Count")
  dev.off()

In [ ]:
# 2.2 Price vs Room Type
png("price_vs_room_type.png", width = 800, height = 600)
ggplot(listings, aes(x = room_type, y = price)) +
  geom_boxplot(fill = "lightgreen") +
  theme_minimal() +
  ggtitle("Price vs Room Type") +
  ylab("Price ($)")
dev.off()

In [ ]:
# 2.3 Price vs Neighbourhood
png("price_vs_neighbourhood.png", width = 1000, height = 600)
ggplot(listings, aes(x = neighbourhood, y = price)) +
  geom_boxplot(fill = "lightpink") +
  theme_minimal() +
  ggtitle("Price vs Neighbourhood") +
  ylab("Price ($)") +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))
dev.off()


In [ ]:
# 2.4 Correlation Heatmap (numeric features)
num_features <- listings[, sapply(listings, is.numeric)]
cor_matrix <- cor(num_features, use = "complete.obs")

png("correlation_heatmap.png", width = 1000, height = 800)
corrplot(cor_matrix, method = "color", type = "upper", 
         tl.cex = 0.8, tl.col = "black", addCoef.col = "black")
dev.off()

# 2.5 Feature importance using Random Forest
set.seed(123)
rf_model <- randomForest(price ~ ., data = listings_encoded, ntree = 100, importance = TRUE)

# Save RF feature importance plot
png("rf_feature_importance.png", width = 1000, height = 600)
varImpPlot(rf_model)
dev.off()

In [ ]:
# -------------------------------
# 3) MODEL BUILDING & TRAINING
# -------------------------------

# 3.1 Train-test split already done
# train_data, test_data from previous steps

# 3.2 Linear Regression
lm_model <- lm(price ~ ., data = train_data)
summary(lm_model)

# Predictions
lm_pred <- predict(lm_model, newdata = test_data)

# RMSE
lm_rmse <- sqrt(mean((test_data$price - lm_pred)^2))
print(paste("Linear Regression RMSE:", round(lm_rmse, 2)))

In [ ]:
# 3.3 Random Forest Regression
rf_model2 <- randomForest(price ~ ., data = train_data, ntree = 200, mtry = 5)
rf_pred <- predict(rf_model2, newdata = test_data)
rf_rmse <- sqrt(mean((test_data$price - rf_pred)^2))
print(paste("Random Forest RMSE:", round(rf_rmse, 2)))

# Hyperparameter tuning (Random Forest)
rf_grid <- expand.grid(mtry = c(3, 5, 7))
train_control <- trainControl(method = "cv", number = 5)
rf_tuned <- train(price ~ ., data = train_data, method = "rf",
                  tuneGrid = rf_grid, trControl = train_control, ntree = 200)
print(rf_tuned)

In [ ]:
# 3.4 XGBoost Regression
# Prepare matrix for XGBoost
train_matrix <- xgb.DMatrix(data = as.matrix(train_data[, -which(names(train_data) == "price")]),
                            label = train_data$price)
test_matrix <- xgb.DMatrix(data = as.matrix(test_data[, -which(names(test_data) == "price")]),
                           label = test_data$price)

# Train XGBoost model
xgb_model <- xgboost(data = train_matrix, max.depth = 6, eta = 0.1, nrounds = 100,
                     objective = "reg:squarederror", verbose = 0)

In [ ]:
# Predictions
xgb_pred <- predict(xgb_model, test_matrix)
xgb_rmse <- sqrt(mean((test_data$price - xgb_pred)^2))
print(paste("XGBoost RMSE:", round(xgb_rmse, 2)))

# Feature importance for XGBoost
importance_matrix <- xgb.importance(feature_names = colnames(train_data[, -which(names(train_data) == "price")]),
                                    model = xgb_model)

# Save XGBoost importance plot
png("xgb_feature_importance.png", width = 1000, height = 600)
xgb.plot.importance(importance_matrix)
dev.off()

In [ ]:
# -------------------------------
# 4) MODEL EVALUATION & INTERPRETATION
# -------------------------------

library(Metrics)   # for MAE
library(caret)     # for R2
library(SHAPforxgboost)

# ---- 4.1 Evaluation Metrics ----
# MAE
lm_mae <- mae(test_data$price, lm_pred)
rf_mae <- mae(test_data$price, rf_pred)
xgb_mae <- mae(test_data$price, xgb_pred)

# R-squared
lm_r2 <- R2(lm_pred, test_data$price)
rf_r2 <- R2(rf_pred, test_data$price)
xgb_r2 <- R2(xgb_pred, test_data$price)

# Print results
cat("\n--- Model Performance ---\n")
cat(sprintf("Linear Regression -> RMSE: %.2f | MAE: %.2f | R²: %.3f\n", lm_rmse, lm_mae, lm_r2))
cat(sprintf("Random Forest     -> RMSE: %.2f | MAE: %.2f | R²: %.3f\n", rf_rmse, rf_mae, rf_r2))
cat(sprintf("XGBoost          -> RMSE: %.2f | MAE: %.2f | R²: %.3f\n", xgb_rmse, xgb_mae, xgb_r2))

# ---- 4.2 ±10-15% Accuracy Check ----
within_10 <- mean(abs((xgb_pred - test_data$price) / test_data$price) <= 0.10) * 100
within_15 <- mean(abs((xgb_pred - test_data$price) / test_data$price) <= 0.15) * 100

cat(sprintf("\nXGBoost: %.1f%% of predictions are within ±10%% of actual price", within_10))
cat(sprintf("\nXGBoost: %.1f%% of predictions are within ±15%% of actual price\n", within_15))

# ---- 4.3 SHAP Values for Interpretability ----
shap_values <- shap.values(xgb_model, X_train = as.matrix(train_data[, -which(names(train_data) == "price")]))
shap_long <- shap.prep(shap_contrib = shap_values$shap_score,
                       X_train = as.matrix(train_data[, -which(names(train_data) == "price")]))

# Top features SHAP summary plot
png("xgb_shap_summary.png", width = 1000, height = 600)
shap.plot.summary(shap_long)
dev.off()

cat("\nSHAP summary plot saved as xgb_shap_summary.png (top features influencing price).")
